# Post Trade Analysis

Run all cells to get an idea of the trade result.

In [ ]:
# Enable autoreload extension
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

from analysis.scripts.database_tool import DatabaseTool
from analysis.scripts.ml_tool import MLTool
from analysis.scripts.visualization_tool import VisualizationTool

#db_tool = DatabaseTool("/home/hao/train_0120-0124.sqlite")
db_tool = DatabaseTool("/tmp/replay.sqlite")
vis_tool = VisualizationTool(db_tool)
ml_tool = MLTool(db_tool)

# Visualization

In [ ]:
vis_tool.draw().show()

# Statistical Analysis

In [ ]:
summary = pd.DataFrame.from_dict(ml_tool.get_summary(), orient="index", columns=["Count"])
summary

In [ ]:
pd.options.display.float_format = '{:.5f}'.format

trade_result = db_tool.load_trade_result()
score_details = MLTool.wrangle(trade_result)
score_details["score"] = trade_result["opportunity.score"]
score_details["profit"] = trade_result["profit"]
score_details["time"] = trade_result["opportunity.bull_flag_pattern.bull_flag.start_time"]
score_details

## Relationship between score and profit

In [ ]:
score_details.select_dtypes("number").corr()

In [ ]:
import seaborn as sns
sns.scatterplot(score_details, x="score", y="profit")

# Machine Learning Experimental

In [ ]:
model, cm = ml_tool.train("random_forest", drop_columns=[])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Not Profitable", "Profitable"], yticklabels=["Not Profitable", "Profitable"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
test_df = MLTool.predict("/tmp/replay.sqlite", model, drop_columns=[])
test_df